# Grad-CAM++ Visualization for MagniNet

This notebook generates Grad-CAM++ overlays for trained MagniNet models.
It supports binary (benign vs malignant) and multi-class explanations.

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from models import MagniNet
from utils.checkpoints import load_checkpoint
from interpretability.gradcam_pp import GradCAMPlusPlus

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
checkpoint = 'runs/magninet_40x/best.pt'
image_path = 'sample.jpg'

model = MagniNet(pretrained_backbones=False).to(device)
load_checkpoint(model, checkpoint, device)
model.eval()

In [ ]:
img = Image.open(image_path).convert('RGB').resize((224,224))
x = torch.from_numpy(np.array(img, dtype=np.float32)/255.0)
x = x.permute(2,0,1).unsqueeze(0).to(device)

In [ ]:
campp = GradCAMPlusPlus(model, target_layer=model.swin_proj, device=device)
cam = campp.generate(x, mode='binary')[0].cpu().numpy()
overlay = campp.overlay_on_image(np.array(img), cam)

plt.figure(figsize=(6,6))
plt.imshow(overlay)
plt.axis('off')
plt.title('Grad-CAM++ (Binary)')
plt.show()

campp.remove_hooks()